In [ ]:
import pandas as pd
import math
import csv
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pylab as plt
import csv

from scipy.stats import gaussian_kde
from scipy.stats import powerlaw

In [ ]:
df = pd.read_csv('allgraph.csv', sep='\t', header=None, encoding='utf-8', engine='python')

In [ ]:
graph = df.rename(columns={0:'KEYWORD',1:'POSITION',2:'SERP'})

In [ ]:
weight = {1:1,2:1,3:1,4:0.85,5:0.6,6:0.5,7:0.5,8:0.3,9:0.3,10:0.2,11:0.15,12:0.15,13:0.10,14:0.10,15:0.08,16:0.05,17:0.03,18:0.02,19:0.01,20:0.009}
graph['POSITION'] = graph['POSITION'].replace(weight, regex=True)

In [ ]:
set_k = set(graph['KEYWORD'])
set_s = set(graph['SERP'])

In [ ]:
authority = {}
hub = {}

for elem in set_k:
    hub[elem] = 1
    
for elem in set_s:
    authority[elem] = 0

In [ ]:
#calcolo score authority (influenzato dalla posizione della serp rispetto la keyword)

for row in graph.itertuples(index=True, name='Pandas'):
    serp = row.SERP
    pos=row.POSITION
    key = row.KEYWORD
    authority[serp] = authority[serp] + pos

In [ ]:
#calcolo score hub (influenzato dallo score dell'authority a cui esso punta)

for row in graph.itertuples(index=True, name='Pandas'):
    serp = row.SERP
    key = row.KEYWORD
    hub[key] = hub[key] + authority[serp]

In [ ]:
#Normalizzazione dei vettori contenete i pesi per hub e authority. Dividendo i vettori, componente per componente, per la radice della sommatoria dei quadrati di
#tutti i loro elementi così che la loro somma dei quadrati sia unitaria

tot_h = 0

for key in hub.keys():
    tot_h = tot_h + (hub[key]**2)
    
tot_h = math.sqrt(tot_h)

for key in hub.keys():
    hub[key] = "%.8f" % (hub[key]/tot_h)

In [ ]:
tot_a = 0

for key in authority.keys():
    tot_a = tot_a + (authority[key]**2)

tot_a = math.sqrt(tot_a)

for key in authority.keys():
    authority[key] = "%.8f" % (authority[key]/tot_a)

In [ ]:
for elem in authority.keys():
    if(elem == 'lifegate.it'):
        print(authority[elem])

In [ ]:
for elem in hub.keys():
    if(elem == 'spiedini'):
        print(hub[elem])

In [ ]:
authority_max = max(authority.keys(), key=(lambda k: authority[k]))
authority_min = min(authority.keys(), key=(lambda k: authority[k]))

hub_max = max(hub.keys(), key=(lambda k: hub[k]))
hub_min = min(hub.keys(), key=(lambda k: hub[k]))

print("MAX AUTHORITY: ",authority_max,"  HITS SCORE:",authority[authority_max])
print("MIN AUTHORITY: ",authority_min,"  HITS SCORE:",authority[authority_min])

print("MAX HUB: ",hub_max,"  HITS SCORE:",hub[hub_max])
print("MIN HUB: ",hub_min,"  HITS SCORE:",hub[hub_min])

In [ ]:
df=[]
for elemen in authority.keys():
    b=authority[elemen]
    df.append(b)

In [ ]:
myarray = np.asarray(df)

In [ ]:
myarray= myarray.astype(float)

In [ ]:
#il grafico mostra la distribuzione dei pesi per le authority

plt.plot(myarray)
plt.show()

In [ ]:
h=sorted(myarray)
fit = stats.norm.pdf(h, np.mean(h), np.std(h))  #this is a fitting indeed

plt.plot(h,fit,'-o')

plt.hist(h,normed=True)      #use this to draw histogram of your data

plt.show() 

In [ ]:
#otteniamo il numero ottimale di bin applicando la regola di Sturge (1+(3.322*log(N)))

nbin=1+3.322*(math.log(len(myarray)))

In [ ]:
nbin=int(nbin)

In [ ]:
index = range(1,72813)
columns = ['auth']


# Option 1: Set the column names in the structured array's dtype 

dtype = [('a','float32')]
values = myarray
df = pd.DataFrame(values, index=index)

In [ ]:
#viene aggiunta una colonna 'binned' per indicare il numero di bin a cui la serp appartiene

df['binned'] = pd.cut(df[0], nbin,labels=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38])

In [ ]:
df['binned'].unique()

In [ ]:
df['serps'] = set_s

In [ ]:
df.loc[df['binned'] == 7]

In [ ]:
#Dizionario di dizionari
#KEY: bin index   VALUE: Dizionario SERP,KEYWORDS
index_serps = {}

#Dizionario contenete l'intersezione delle keyword collegate a serp appartenenti allo stesso cluster
#KEY: bin index   VALUE: Keywords intersection
index_key_set = {}

for i in range(2,39):
    start_time = time.time()
    if(not df.loc[df['binned'] == i].empty):
        df_t = df.loc[df['binned'] == i]
        key_serp = {}
        list_key = []
        for row in df_t.itertuples(index=True, name='Pandas'):
            serp = row.serps
            
            #considera solo le keyword che si trovano nelle prime 5 posizioni per poter caratterizzare la serp relativa come top site
            df_k = graph.loc[graph['SERP']== serp].loc[graph['POSITION'] >= 0.6] 
            
            #lista contenente i set delle keyword per ogni set, utilizzata successivamente per ottenere l'intersezione
            list_key.append(set(df_k['KEYWORD']))
            key_serp[serp] = set(df_k['KEYWORD'])
        
        #per ogni bin i contiene la coppia serp,keywords
        index_serps[i] = key_serp 
        
        #contiene l'intersezione delle keyword relative al bin i
        index_key_set[i] = set.intersection(*list_key) 
        
    print("Iterazione " , i ,"  --- %s seconds ---" % (time.time() - start_time))

In [ ]:
str(index_serps.keys())

In [ ]:
for i in list(index_serps.keys()):
    print("BIN: ", i)
    print("SERPS: ", list(index_serps[i].keys()))
    print("N.KEYWORDS: ", len(index_key_set[i]))
    print("-------------------------------------------------------------------------")

In [ ]:
index_key_set[38].intersection(index_key_set[16])

In [ ]:
#scrive in diversi file csv la lista di keyword raggruppate per ogni bin

for i in list(index_serps.keys()):
    path =  "Clusters/ListKeywordsForBin_"
    est = ".csv"
    csvFilename = (path+str(i)+est)  # what name you want to save your csv as
    csv = open(csvFilename, "w", encoding = "utf-8")  # create or open csv, "w" to write strings
    csv.write('')
    for elem in index_key_set[i]:
        csv.write(elem + "\n")

In [ ]:
index = [4,7,8,10,11,13,16,17,19,25,29,38]  #non consideriamo 2,3,6 poichè non possiedono keyword al loro interno

In [ ]:
str_113 = ("buonissimo.org buttalapasta.it cucchiaio.it")
str_679 = ("cookaround.com lacucinaitaliana.it giallozafferano.it salepepe.it")

In [ ]:
lst_key_index_113 = []
lst_key_index_679 = []

j = 0
for elem in set_101113:
    if(j<=15):
        line = (str_113+","+elem)
        lst_key_index_113.append(line)
        j = j+1

k = 0
for elem in set_161719:
    if(k<=15):
        line = (str_679+","+elem)
        lst_key_index_679.append(line)
        k = k+1

In [ ]:
#csv contenente l'intersezione tra le keyword appartenenti ai bin 10,11,13

csvFilename = "graphSample_113.csv"
csv = open(csvFilename, "w", encoding = "utf-8")  # create or open csv, "w" to write strings
csv.write('')
for elem in lst_key_index_113:
        csv.write(elem + "\n") 

In [ ]:
#csv contenente l'intersezione tra le keyword appartenenti ai bin 16,17,19

csvFilename = "graphSample_679.csv"
csv = open(csvFilename, "w", encoding = "utf-8")  # create or open csv, "w" to write strings
csv.write('')
for elem in lst_key_index_679:
        csv.write(elem + "\n") 

In [ ]:
#dizionario serps / keywords
#KEY: serps di ogni bin ottenuto  VALUE: keywords in comune tra le serp di ogni bin

serps_conctn = {}
for i in index:
    serps_conctn[i] = ""
    for elem in list(index_serps[i].keys()):
        if(serps_conctn[i] == ""):
            serps_conctn[i] = (elem+" ")
        else:
            serps_conctn[i] = (serps_conctn[i]+elem+" ")

In [ ]:
#crea una lista serps,keywords al fine di realizzare un sample del grafo per la visualizzazione

lst_key_index = []
for i in index:
    j = 0
    for elem in index_key_set[i]:
        if(j<=20):
            line = (serps_conctn[i]+","+elem)
            lst_key_index.append(line)
            j = j+1

In [ ]:
csvFilename = "graphSample.csv"
csv = open(csvFilename, "w", encoding = "utf-8")  # create or open csv, "w" to write strings
csv.write('')
for elem in lst_key_index:
        csv.write(elem + "\n") 